# Monad Types in JavaScript

Based on [Understanding Monads](https://en.wikibooks.org/wiki/Haskell/Understanding_monads) in Haskell.

A monad is defined by three things:

* a [type constructor](https://en.wikibooks.org/wiki/Haskell/More_on_datatypes#Parameterized_Types) `m` (does not exist in JavaScript)
* a function `return`
* an operator `(>>=)` which is pronounced "bind"

The function and operator are methods of the Monad class:

In [1]:
"use strict"
class Monad {
    constructor(x) {
        this.x = x
    }

    /**
     * return :: a -> m a
     * @param {*} x
     * @returns {Monad}
     */
    static return(x) {
        throw new Error('derived types must implement return') 
    }

    /**
     * bind :: m a -> (a -> m b) -> m b
     * @abstract
     * @param {function} g
     * @returns {Monad}
     */
    bind(g) {
        throw new Error('derived types must implement bind')
    }

    /**
     * then :: m a -> m b -> m b
     * @param {Monad} m
     * @returns {Monad}
     */
    then(m) {
        return this.bind(_ => m)
    }
}

[Function: Monad]

In [2]:
"use strict"
class Maybe extends Monad {
    /**
     * bind :: Maybe a -> (Maybe -> Maybe b) -> Maybe b
     * @override
     * @param {function} g
     * @returns {Maybe}
     */
    bind(g) {
        if (this instanceof Something)
            return g(this.x)
        else
            return Nothing.return()
    }
}

class Something extends Maybe {
    /**
     * return :: a -> Something a
     * @override
     * @param {*} x
     * @returns {Something}
     */
    static return(x) {
        return new Something(x)
    }
}

class Nothing extends Maybe {
    /**
     * return :: Nothing
     * @override
     * @returns {Nothing}
     */
    static return() {
        return new Nothing(null)
    }
}

[Function: Nothing]

In [3]:
m = Something.return(1)
m.bind(x => Something.return(x + 1))
 .bind(x => Something.return(x + 1))

Something { x: 3 }

In [4]:
m.bind(x => Nothing.return())
 .bind(x => Something.return(x + 1))

Nothing { x: null }

## Monad Laws

Every instance of the Monad class (and thus all implementations of bind (>>=) and return) must obey the following three laws:

```haskell
m >>= return     =  m                        -- right unit
return x >>= f   =  f x                      -- left unit

(m >>= f) >>= g  =  m >>= (\x -> f x >>= g)  -- associativity
```

In [5]:
(function() {
    "use strict"
    const equal = require('assert').deepEqual

    const m = new Something(42)
    const ret = Something.return
    const f = (x => ret(x * x))
    const g = (x => ret(-x))

    // right unit
    equal(m.bind(ret), m)

    // left unit
    equal(ret(42).bind(f), f(42))

    // associativity
    equal(m.bind(f).bind(g), m.bind(x => f(x).bind(g)))

    return 'ok'
}())

'ok'